In [159]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import MaxPooling2D, Dense, Flatten, Conv2D
from keras import backend as K
import glob

In [160]:
img_height=80
img_width=80
channel=3
X_train=np.empty([8005, img_height, img_width, channel], np.int32)
X_test=np.empty([2023, img_height, img_width, channel], np.int32)
y_train=np.full([8005,], fill_value=5)
y_test=np.full([2023,], fill_value=6)
i = 0
# cat = 0 , dog =1

In [161]:
for filepath in glob.iglob('training_set/cats/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_train[i] = 0
    X_train[i] = img
    i = i + 1
    
for filepath in glob.iglob('training_set/dogs/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_train[i] = 1
    X_train[i] = img
    i = i + 1    
    
i=0    
    
for filepath in glob.iglob('test_set/cats/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_test[i] = 0
    X_test[i] = img
    i = i + 1
    
for filepath in glob.iglob('test_set/dogs/*.jpg'):
    img=Image.open(filepath)
    img=img.resize((img_height,img_width), Image.ANTIALIAS)
    img=np.array(img)
    y_test[i] = 1
    X_test[i] = img
    i = i + 1      

In [162]:
X_train = X_train/255
X_test  = X_test/255
y_train = keras.utils.to_categorical(y_train, 2)
y_test  = keras.utils.to_categorical(y_test, 2)
input_shape=(img_height, img_width, channel)

In [208]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2,2), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [209]:
model.fit(X_train, y_train, batch_size=128, epochs=10, verbose=1, validation_data=(X_test, y_test))
score=model.evaluate(X_test, y_test)
print('Loss:', score[0])
print('Accuracy:',score[1])

Train on 8005 samples, validate on 2023 samples
Epoch 1/10
8005/8005 [==============================] - 30s 4ms/step - loss: 0.6708 - accuracy: 0.5843 - val_loss: 0.6509 - val_accuracy: 0.6110
Epoch 2/10
8005/8005 [==============================] - 29s 4ms/step - loss: 0.6053 - accuracy: 0.6701 - val_loss: 0.5783 - val_accuracy: 0.7098
Epoch 3/10
8005/8005 [==============================] - 31s 4ms/step - loss: 0.5484 - accuracy: 0.7269 - val_loss: 0.5391 - val_accuracy: 0.7459
Epoch 4/10
8005/8005 [==============================] - 35s 4ms/step - loss: 0.5110 - accuracy: 0.7567 - val_loss: 0.5207 - val_accuracy: 0.7563
Epoch 5/10
8005/8005 [==============================] - 36s 4ms/step - loss: 0.4777 - accuracy: 0.7766 - val_loss: 0.5278 - val_accuracy: 0.7464
Epoch 6/10
8005/8005 [==============================] - 35s 4ms/step - loss: 0.4656 - accuracy: 0.7791 - val_loss: 0.5975 - val_accuracy: 0.7177
Epoch 7/10
8005/8005 [==============================] - 35s 4ms/step - loss: 0.464

In [211]:
model.save('cat_dog.h5')

This model achieved an accuracy of 77%. Data augmentation can probably be used to improve accuracy.